# Tugas UTS Web Mining
Nama : M Nur Fauzan W || NIM  : 190411100064

## 1. Membuat Analisa Clustring dengan K-Means

In [1]:
# !git clone --depth=1 https://github.com/twintproject/twint.git
# %cd twint
# !pip3 install . -r requirements.txt
# !pip install twint
# !pip install aiohttp==3.7.0
# !pip install nest_asyncio

In [2]:
import twint
import pandas as pd
import re
import numpy as np
import nltk
# nltk.download('punkt')
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nest_asyncio
nest_asyncio.apply()

# c = twint.Config()
# c.Search = 'tragedi kanjuruhan'
# c.Pandas = True
# c.Limit = 60
# c.Store_csv = True
# c.Custom["tweet"] = ["tweet"]
# c.Output = "datatweet.csv"
# twint.run.Search(c)

In [3]:
data = pd.read_csv('datatweet.csv')
data

,tweet
0,TGIPF Tragedi Kanjuruhan Serahkan Hasil Invest...
1,Sejumlah Polisi Malang Sujud Massal Mohon Maaf...
2,Korban Meninggal dalam Tragedi Kanjuruhan Bert...
3,"Berjiwa Besar, Dirut PT LIB Pastikan Siap Bert..."
4,#Surabaya Kira-kira 100 anggota Polis Kota Mal...
5,Ya Tuhan mohon kesembuhan bagi teman2 yang jad...
6,buat isilop isilop yg bilang tragedi kanjuruha...
7,Irjen Dedi Disentil Anggota DPR Abis Buat Pern...
8,Korban tewas Tragedi Kanjuruhan bertambah menj...
9,Komnas HAM Mengaku Sudah Kantongi Rekaman Vide...


In [4]:
def remove_stopwords(text):
    with open('stopwords.txt') as f:
        stopwords = f.readlines()
        stopwords = [x.strip() for x in stopwords]
    text = nltk.word_tokenize(text)
    text = [word for word in text if word not in stopwords]      
    return text

def stemming(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    result = [stemmer.stem(word) for word in text]
    return result

def preprocessing(text):
    #case folding
    text = text.lower()

    #remove non ASCII (emoticon, chinese word, .etc)
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\'," ").replace('\\f'," ").replace('\\r'," ")

    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')

    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    text = ' '.join(re.sub("([0-9])","", text).split())

    #replace weird characters
    text = text.replace('“', '"')
    text = text.replace('”', '"')
    text = text.replace('-', ' ')

    #tokenization and remove stopwords
    text = remove_stopwords(text)

    #remove punctuation    
    text = [''.join(c for c in s if c not in string.punctuation) for s in text]  

    #stemming
    text = stemming(text)

    #remove empty string
    text = list(filter(None, text))
    return text

In [5]:
data['tweet'].apply(preprocessing).to_csv('preprocessing_tweet.csv')

KeyboardInterrupt: 

In [59]:
pd.read_csv('preprocessing_tweet.csv')

In [44]:
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer
dataTextPre = pd.read_csv('preprocessing_tweet.csv')
vectorizer = CountVectorizer(min_df=1)
bag = vectorizer.fit_transform(dataTextPre['tweet'])

matrik_vsm=bag.toarray()
matrik_vsm.shape

(60, 386)

In [45]:
matrik_vsm[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [46]:
a=vectorizer.get_feature_names_out()

In [47]:
dataTF =pd.DataFrame(data=matrik_vsm,index=list(range(1, len(matrik_vsm[:,1])+1, )),columns=[a])
dataTF

,abis,acara,ade,afinta,air,aju,akhirat,akibat,aktor,aku,...,warga,waspada,week,westlife,wib,widyagama,wilayah,ya,yg,zat
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
7,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
8,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [48]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3).fit(dataTF)
prediksi = kmeans.predict(dataTF)
centroids = kmeans.cluster_centers_

c:\Users\M Nur Fauzan W\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
c:\Users\M Nur Fauzan W\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(


In [49]:
prediksi

array([0, 0, 2, 0, 0, 1, 1, 1, 2, 0, 1, 0, 2, 1, 0, 1, 0, 0, 0, 2, 2, 0,
       0, 2, 1, 0, 0, 0, 0, 2, 2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1])

In [73]:
cluster = pd.DataFrame(prediksi,columns=['Cluster'])
data = pd.read_csv('preprocessing_tweet.csv')
cluster

,Cluster
0,0
1,0
2,2
3,0
4,0
5,1
6,1
7,1
8,2
9,0


## 2. Membuat Ringkasan Dokumen Dari Berita Online Dengan Metode PageRank

In [51]:
import requests
import pandas as pd
import pandas as pd
import re
import numpy as np
import nltk
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from bs4 import BeautifulSoup as bs
URL = "https://news.detik.com/berita/d-6334129/5-fakta-baru-tragedi-kanjuruhan-6-tersangka-hingga-soal-gas-air-mata?single=1"
resp = requests.get(URL)
soup = bs(resp.text, "lxml")
elements = soup.find("div", attrs={"class":"detail__body-text itp_bodycontent"}).find_all('p')
dokumen = []
split_kalimat = [] 
for i,paragraf in enumerate(elements):
    dokumen.append(paragraf.text)
    # print(kalimat)
    for j,kalimat in enumerate(dokumen[i].split(".")):
        # print(len(kalimat))
        if len(kalimat) == 0:
            continue
        else:
            split_kalimat.append(kalimat)
def remove_stopwords(text):
    with open('stopwords.txt') as f:
        stopwords = f.readlines()
        stopwords = [x.strip() for x in stopwords]
    text = nltk.word_tokenize(text)
    text = [word for word in text if word not in stopwords]              
    return text

def stemming(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    result = [stemmer.stem(word) for word in text]
    return result

def preprocessing(kalimat):
    import re
    res_kata = []
    for j,kata in enumerate(kalimat.split(" ")):
        kata = kata.lower()
        kata = kata.replace(",", "")
        kata = kata.replace("\r\n","")
        kata = kata.replace("\"","")
        kata = ''.join(re.sub("[0-9]","",kata))
        if len(kata) > 0:
            res_kata.append(kata)
    # print(text)
    return res_kata

df = pd.DataFrame(split_kalimat,columns=['kata'])
df['kata'].apply(preprocessing).to_csv("preprocessing_summary.csv")
data = pd.read_csv("preprocessing_summary.csv")
data


    


,Unnamed: 0,kata
0,0,"['fakta-fakta', 'baru', 'terkait', 'tragedi', ..."
1,1,"['mulai', 'dari', 'ada', 'enam', 'tersangka', ..."
2,2,"['berikut', 'fakta', 'terbaru', 'tragedi', 'ka..."
3,3,"['kapolri', 'jenderal', 'listyo', 'sigit', 'pr..."
4,4,"['direktur', 'utama', 'pt', 'lib', 'akhmad', '..."
5,5,['advertisement']
6,6,"['scroll', 'to', 'resume', 'content']"
7,7,"['berdasarkan', 'gelar', 'dan', 'alat', 'bukti..."
8,8,"['lima', 'tersangka', 'lainnya', 'ialah', 'kab..."
9,9,"['kapolri', 'menyebut', 'pt', 'liga', 'indones..."


In [52]:
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer
dataTextPre = pd.read_csv('preprocessing_summary.csv')
vectorizer = CountVectorizer(min_df=1)
bag = vectorizer.fit_transform(dataTextPre['kata'])

In [53]:
matrik_vsm=bag.toarray()
matrik_vsm.shape

(55, 308)

In [54]:
matrik_vsm[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [55]:
a=vectorizer.get_feature_names_out()

In [56]:
dataTF =pd.DataFrame(data=matrik_vsm,index=list(range(1, len(matrik_vsm[:,1])+1, )),columns=[a])
dataTF

,achmadi,ada,adalah,adanya,advertisement,agar,ah,air,akan,akhmad,...,utara,verifikasi,versus,video,wahyu,waktu,waktunya,wib,yaitu,yang
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
6,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [57]:
import matplotlib.pyplot as plt
import networkx as nx
from nltk.probability import DictionaryConditionalProbDist
data = pd.read_csv("preprocessing_summary.csv")
G = nx.DiGraph()
G = nx.from_pandas_edgelist(data,edge_key='',edge_attr='kata',create_using=nx.Graph())
nx.draw(G)
plt.show()
# dataTF.

KeyError: 'source'